In [2]:
import pytest
import ipytest
ipytest.autoconfig()

function1 ---> connect to buckect and retrieve the objects from the bucket


In [3]:
import boto3
import pyarrow.parquet as pq
import pandas as pd
bucket = 'rand1'
s3_client = boto3.client("s3")
with open("./load_test_db/hello_test.parquet", "rb") as f:
    s3_client.upload_fileobj(f, f"{bucket}", "hello.parquet")
bucket_objects = s3_client.list_objects_v2(Bucket = bucket)
assert bucket_objects['KeyCount'] == 1
file_paths = [f's3://{bucket}/{obj["Key"]}' for obj in bucket_objects['Contents']]
print(file_paths)
for file_path in file_paths:
    s3_file = pq.ParquetDataset(file_path)
    table = s3_file.read().to_pandas()
    df= pd.DataFrame(table)
    print(df)
assert df.columns.tolist() == ['hello']

['s3://rand1/hello.parquet']
   hello
0  world


In [4]:
import boto3
def retrive_files(bucket):
    s3_client = boto3.client("s3")
    bucket = s3_client.list_objects_v2(Bucket = bucket)
    return bucket


In [13]:
%%ipytest
from moto import mock_s3
import pytest
import pyarrow.parquet as pq
import pg8000
from pyarrow import fs

@pytest.fixture(scope='function')
def aws_credentials():
    """Mocked AWS Credentials for moto."""
    os.environ['AWS_ACCESS_KEY_ID'] = 'test'
    os.environ['AWS_SECRET_ACCESS_KEY'] = 'test'
    os.environ['AWS_SECURITY_TOKEN'] = 'test'
    os.environ['AWS_SESSION_TOKEN'] = 'test'
    os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

@mock_s3
def test_creating_mock_s3():
    conn = boto3.client("s3", region_name="us-east-1")
    res = conn.create_bucket(Bucket="test_bucket_29")
    assert res['ResponseMetadata']['HTTPStatusCode'] == 200

@mock_s3
def test_accessing_the_objects_in_the_bucket():
    conn = boto3.client("s3", region_name="us-east-1")
    res = conn.create_bucket(Bucket="test_bucket_29")
    parquet_files = retrive_files('test_bucket_29')
    assert parquet_files['KeyCount'] == 0

@mock_s3
def test_retreiving_objects_in_the_bucket():
    bucket = "test_bucket_29"
    conn = boto3.client("s3")
    res = conn.create_bucket(Bucket=bucket)
    parquet_files = retrive_files(bucket)
    # uploading some fake data to bucket
    with open("./load_test_db/hello_test.parquet", "rb") as f:
        conn.upload_fileobj(f, f"{bucket}", "hello.parquet")
    parquet_files = retrive_files(bucket)
    assert parquet_files['KeyCount'] == 1
    
@pytest.fixture
def test_retreiving_objects_in_the_bucketss():
    bucket ="rand2"
    s3_client = boto3.client("s3")
    res = s3_client.create_bucket(Bucket=bucket,CreateBucketConfiguration={'LocationConstraint': 'eu-west-2'})
    with open("./load_test_db/hello_test.parquet", "rb") as f:
        s3_client.upload_fileobj(f, f"{bucket}", "hello.parquet")
    bucket_objects = retrive_files(bucket)
    file_paths = [f's3://{bucket}/{obj["Key"]}' for obj in bucket_objects['Contents']]
    for file_path in file_paths:
        s3_file = pq.ParquetDataset(file_path)
        table = s3_file.read().to_pandas()
        df= pd.DataFrame(table)
    assert df.columns.tolist() == ['hello']

# @pytest.fixture
# def test_warehouse_connection():
#     connect = make_warehouse_connection()
#     assert isinstance(connect, pg8000.Connection)  


...                                                                                          [100%]
3 passed in 1.25s


In [6]:
import os
from dotenv import load_dotenv
from os.path import join, dirname
from pathlib import Path
import pg8000
def make_warehouse_connection():
    # dotenv_path = join(dirname(__file__), '../config/.env.data_warehouse')
    dotenv_path = Path('./config/.env.data_warehouse')
    load_dotenv(dotenv_path)
    API_HOST =  os.environ["host"]
    API_USER = os.environ["user"]
    API_PASS = os.environ["password"]
    API_DBASE = os.environ["database"]
    conn = pg8000.connect(
        host=API_HOST,
        user=API_USER,
        password=API_PASS,
        database=API_DBASE
    )
    return conn

print(make_warehouse_connection())
